# Timeline

[![open_in_colab][colab_badge]][colab_notebook_link]
[![open_in_binder][binder_badge]][binder_notebook_link]

[colab_badge]: https://colab.research.google.com/assets/colab-badge.svg
[colab_notebook_link]: https://colab.research.google.com/github/foursquare/fsq-studio-sdk-examples/blob/master/notebooks/05%20-%20Timeline.ipynb
[binder_badge]: https://mybinder.org/badge_logo.svg
[binder_notebook_link]: https://mybinder.org/v2/gh/foursquare/fsq-studio-sdk-examples/master?urlpath=lab/tree/notebooks/05%20-%20Timeline.ipynb

Foursquare Studio supports [time playback for temporal analytics](https://docs.unfolded.ai/studio/map-guide/playback). If you have a column in your dataset with temporal data, you can add a filter to it and it will be displayed as an interactive timeline over the map. Studio Map SDK makes it possible to control this filter remotely.

## Dependencies

This notebook requires the following Python dependencies:

- `foursquare.map-sdk`: The Studio Map SDK
- `pandas`: DataFrame library

If running this notebook in Binder, these dependencies should already be installed. If running in Colab, the next cell will install these dependencies.

In [ ]:
# If in Colab, install this notebook's required dependencies
import sys
if "google.colab" in sys.modules:
    !pip install 'foursquare.map_sdk>=1.0' pandas

## Imports

In [ ]:
from foursquare.map_sdk import create_map
from datetime import datetime
import pandas as pd
from uuid import uuid4

## Using Map Timelines

First, create a map and add data to it:

In [ ]:
unfolded_map = create_map(height=600)
unfolded_map

In [ ]:
url = 'https://4sq-studio-public.s3.us-west-2.amazonaws.com/sdk/examples/sample-data/earthquakes.csv'
df = pd.read_csv(url)
dataset_id = str(uuid4())
unfolded_map.add_dataset({
    'id': dataset_id,
    'label': 'Earthquakes',
    'data': df
})

## Updating timeline range

First, let's convert the `DateTime` column to [`datetime`](https://docs.python.org/3/library/datetime.html):

In [ ]:
df['DateTime']= pd.to_datetime(df['DateTime'])

Now we can calculate the time extent:

In [ ]:
time_extent = [df['DateTime'].min(), df['DateTime'].max()]
time_extent

Here we add a `DateTime` filter to the map:

In [ ]:
unfolded_map.add_filter({
    'id': 'time-filter',
    'sources': [
        {
            'data_id': dataset_id,
            'field_name': 'DateTime'
        }
    ], 
    'value': [
        time_extent[0].timestamp() * 1000,
        time_extent[1].timestamp() * 1000,
    ]
})

Once you execute the above, you should see the timeline appear in the map.

## Controlling the timeline

Once we have added the timeline filter we can use [`update_timeline()`](https://docs.unfolded.ai/map-sdk-v1/api/update-timeline) to change attributes of the timeline itself (separate from the filter). We use the same filter id to refer to it:

In [ ]:
unfolded_map.update_timeline(
    'time-filter',
    time_format='YYYY-MM-DD',
    view='minified',
)

## Animating the timeline

Let's first set the timeline to a narrower range:

In [ ]:
unfolded_map.update_filter(
    'time-filter',
    value = (
        datetime(1967,1,1).timestamp() * 1000,
        datetime(1968,1,1).timestamp() * 1000
    )
)

Now we can start the animation:

In [ ]:
unfolded_map.update_timeline(
    'time-filter',
    is_animating=True,
    animation_speed=1
)

We can also take the timeline off the map and put it in the side panel:

In [ ]:
unfolded_map.update_timeline(
    'time-filter',
    view='side',
)

Now let's stop the animation and display the timeline back on the main map:

In [ ]:
unfolded_map.update_timeline(
    'time-filter',
    is_animating=False,
    view='enlarged',
)